In [1]:
import os
import matplotlib.pyplot as plt
import numpy as np
from astropy.io import fits
from astropy.wcs import WCS
from a345_utilities import print_header    
from matplotlib.patches import Rectangle as rect 
import numpy as np
import os, time
from photutils.detection import DAOStarFinder
from astropy.stats import mad_std
from photutils.aperture import aperture_photometry, CircularAperture
from photutils.aperture import CircularAnnulus
from scipy.optimize import curve_fit
import warnings
warnings.filterwarnings('ignore')
from astropy.stats import sigma_clipped_stats
import pandas as pd
from astroquery.astrometry_net import AstrometryNet
from astroquery.exceptions import TimeoutError


In [59]:

RADIUS = 10


def error_func(star,band:str,radius=RADIUS, six_x_six=False):

    '''
    Function is to calculate the errors on a giving set of magnitudes
    The function takes in a set of images and for each image finds the error 
    associated on that magnitude.
    '''
    
    if six_x_six == True:
        path_data = '/Volumes/external_2T/corrected/6x6/wcs/6x6'
        star = star+'_6x6'
    else:
        path_data =  '/Volumes/external_2T/corrected/wcs'
    star_list = os.listdir(path_data+'/'+star+'/'+band)       
    
    for img in star_list:

        if not img.endswith('.fits'):          
            continue

        with fits.open(path_data+'/'+star+'/'+band+'/'+img) as hdu:             
            img_header = hdu[0].header
            img_data = hdu[0].data   
        mean, median, std = sigma_clipped_stats(img_data)
        wcs = WCS(img_header)     
        if six_x_six == True:
            
            daofind = DAOStarFinder(fwhm=12, threshold=5*std) 
        else:
            daofind = DAOStarFinder(fwhm=4.2, threshold=4*std)
        sources = daofind(img_data)
        for col in sources.colnames:  
            sources[col].info.format = '%.8g'
        sources.sort('flux')
        sources.reverse()
        #print(img_c)

        #load in calibration stars
        data_cal = np.transpose(np.loadtxt('/Volumes/external_2T/cal_stars/error_cal/'+star[0:11]+'_calibration_stars.txt', skiprows=1, delimiter=","))
        id_cal=data_cal[0]
        mag_cal_G=data_cal[3]
        ra_cal=data_cal[1]
        dec_cal=data_cal[2]
        mag_err_G = data_cal[4]
        mag_cal_I = data_cal[7]
        mag_err_I =data_cal[8]
        mag_cal_R = data_cal[5]
        mag_err_R = data_cal[6]
        id_cal1 = id_cal[..., None] 
        mag_cal1 = mag_cal_G[..., None] 
        ra_cal1 = ra_cal[..., None] 
        dec_cal1 = dec_cal[..., None] 
        mag_err1 = mag_err_G[...,None]



        r1=radius
        r2=r1+1
        r3=r2+1

        
        #calibration 
        source1_x, source1_y= wcs.wcs_world2pix(ra_cal,dec_cal,1)
        #print(source1_x,source1_y)
        source1 = np.transpose((source1_x, source1_y))
        #print(source1)
        calibration_aperture = CircularAperture(source1, r1)  
        calibration_annulus = CircularAnnulus(source1, r2, r3)
        calibration_phot = [calibration_aperture, calibration_annulus]

        #sources from dao
        source2_x, source2_y =(sources['xcentroid'] , sources['ycentroid'] )

        source2=np.transpose((source2_x, source2_y))
        target_aperture = CircularAperture(source2, r1)  
        target_annulus = CircularAnnulus(source2, r2, r3)
        target_phot = [target_aperture, target_annulus]

        
        
        phot_table_calibration = aperture_photometry(img_data, calibration_phot)
        phot_table_target = aperture_photometry(img_data, target_phot)
        print(calibration_phot)

        zp_arr = []
        for i in np.arange(len(id_cal)):
            #mean counts in the background of the first calibration star
            bkg_mean_cal = float(phot_table_calibration[i]['aperture_sum_1']/calibration_annulus.area) 

            #number of counts in the aperture area of the calibratin star
            bcal = bkg_mean_cal*calibration_aperture.area
            print(phot_table_calibration[i]['aperture_sum_1'])
            #flux of the first calibration star
            cal_flux = float(phot_table_calibration[i]['aperture_sum_0']-bcal)
            print(cal_flux)
            #instrumental magnitude if the calibration star
            
            instrument_mag_cal = -2.5*np.log10((cal_flux))
            # print(instrument_mag_cal)


            zp=mag_cal_G[i]-instrument_mag_cal

            zp_arr.append(zp)
            cal_err = np.sqrt((mag_err_G[i]/mag_cal1[i])**2)
        def line(x,m,c):
            return m*x+c

        pop,pcov = curve_fit(line,id_cal,zp_arr,p0 = [0,1])#,sigma = mag_err_G)
        
        plt.figure(figsize=(12,10))
        plt.scatter(id_cal,zp_arr)
        plt.plot(id_cal,pop[0]*id_cal+pop[1])
        plt.xlabel('Id of the calibration star')
        plt.ylabel('Zero Point of a Star')
        plt.show()
        print(np.sqrt(np.diag(pcov)))

